In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_vcm_dmt.a_leadtime_sto_ambient_internal_rpt 
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
AS
WITH
all_sto AS (
  SELECT
    sto_type,
    sto_number,
    sto_item,
    product_id,
    start_date,
    end_date,
    store_id,
    dc_id,
    ontime_check,
    IF(leadtime_actual IS NULL, NULL, GREATEST(0,leadtime_actual)) AS leadtime_actual,
    leadtime_qual,
    NULL AS do_qty,
    SUM(sto_qty) AS sto_qty,
    SUM(gi_qty) AS gi_qty,
    SUM(infull_qty) AS gi_qty_infull,
    SUM(ontime_qty) AS gi_qty_ontime
  FROM {catalog_name}.udp_wcm_gold_vcm_dmt.a_leadtime_sto_ambient_bs_internal_rpt
  WHERE ontime_check IN ('LATE', 'ONTIME', 'IN LEADTIME REQUIRED')
  GROUP BY 1,2,3,4,5,6,7,8,9,10,11
  UNION ALL
  SELECT
    sto_type,
    sto_number,
    sto_item,
    product_id,
    start_date,
    end_date,
    store_id,
    dc_id,
    ontime_check,
    IF(leadtime_actual IS NULL, NULL, GREATEST(0,leadtime_actual)) AS leadtime_actual,
    leadtime_qual,
    SUM(do_qty) AS do_qty,
    NULL AS sto_qty,
    SUM(gi_qty) AS gi_qty,
    SUM(infull_qty) AS gi_qty_infull,
    SUM(ontime_qty) AS gi_qty_ontime
  FROM {catalog_name}.udp_wcm_gold_vcm_dmt.a_leadtime_sto_ambient_ft_internal_rpt
  WHERE ontime_check IN ('LATE', 'ONTIME', 'IN LEADTIME REQUIRED')
  GROUP BY 1,2,3,4,5,6,7,8,9,10,11
)
SELECT * FROM all_sto
""")